# CME 241 Assignment 9

## Shaan Patel

### Question 1

In [16]:
from abc import abstractmethod
from typing import Dict, Tuple
from rl.chapter9.order_book import DollarsAndShares, PriceSizePairs, OrderBook
from rl.distribution import Distribution, Categorical
from rl.markov_process import MarkovProcess, NonTerminal, State
from numpy.random import poisson, random_sample

In [17]:
class LimitOrder(Distribution[OrderBook]):
    def __init__(self, book: OrderBook):
        self.book = book
    
    @abstractmethod
    def sample(self):
        x = random_sample()
        num_shares = poisson(1)
        if x > 0.5:
            price = self.book.ask_price()*random_sample()
            ret_tuple = self.book.sell_limit_order(price, num_shares)
            return ret_tuple[1]
        else:
            price = 10*random_sample() + self.book.bid_price()
            ret_tuple = self.book.buy_limit_order(price, num_shares)
            return ret_tuple[1]

class MarketOrder(Distribution[OrderBook]):
    def __init__(self, book: OrderBook):
        self.book = book

    def sample(self):
        x = random_sample()
        num_shares = poisson(1)
        if x > 0.5:
            ret_tuple = self.book.sell_market_order(num_shares)
            return ret_tuple[1]
        else:
            ret_tuple = self.book.buy_market_order(num_shares)
            return ret_tuple[1]



class OBDynam(MarkovProcess[OrderBook]):
    def transition(self, state: OrderBook) -> Distribution[OrderBook]:
        x = random_sample()
        if x > 0.5:
            LO = LimitOrder[state]
            return LO
        else:
            MO = MarketOrder[state]
            return MO
        


In [18]:
bids: PriceSizePairs = [DollarsAndShares(
        dollars=x,
        shares=poisson(100. - (100 - x) * 10)
    ) for x in range(100, 90, -1)]
asks: PriceSizePairs = [DollarsAndShares(
        dollars=x,
        shares=poisson(100. - (x - 105) * 10)
    ) for x in range(105, 115, 1)]

ob0: OrderBook = OrderBook(descending_bids=bids, ascending_asks=asks)

dynam = OBDynam()

### Question 2

Given that we have a temporary price impact of:

$$Q_t = P_t (1- \beta N_t - \theta X_t) $$

The optimal value function is:

$$V^*_t((P_t, R_t)) = \max_{N_t} \{N_t P_t(1 - \beta N_t - \theta X_t) + E[V^*_{t+1}((P_{t+1},R_{t+1}))]\} $$

$$V^*_{T-1}((P_{T-1}, R_{T-1})) = N_{T-1} P_{T-1}(1 - \beta N_{T-1} - \theta X_{T-1})$$

$$ = R_{T-1} P_{T-1}(1 - \beta R_{T-1} - \theta X_{T-1}) $$

Therefore we can infer $V^*_{T-2}((P_{T-2}, R_{T-2}))$ as:

$$\max_{N_{T-2}}\{N_{T-2}P_{T-2}(1 - \beta N_{T-2} - \theta X_{T-2}) + E[R_{T-1} P_{T-1}(1 - \beta R_{T-1} - \theta X_{T-1})]\} $$

$$\max_{N_{T-2}}\{N_{T-2}P_{T-2}(1 - \beta N_{T-2} - \theta X_{T-2}) $$
$$+ E[(R_{T-2} - N_{T-2}) P_{T-1}(1 - \beta (R_{T-2} - N_{T-2}) - \theta X_{T-1})]\} $$

This leads to:

$$ \max_{N_{T-2}}N_{T-2}P_{T-2}(1 - \beta N_{T-2} - \theta X_{T-2}) \\
+ E[(R_{T-2} - N_{T-2}) (P_{T-2}e^{Z_{T-2}})(1 - \beta (R_{T-2} - N_{T-2}) - \theta (\rho X_{T-2} + \eta_{T-2}))] $$

The expected value is removed because we know the means of our random variables. By grouping like terms, we get:

$$ \max_{N_{T_2}} N_{T-2}P_{T-2}(1 - e^{\mu_Z + \frac{\sigma_Z^2}{2}}) - \beta N^2_{T-2}P_{T-2} (1 + e^{\mu_Z + \frac{\sigma_Z^2}{2}}) \\
 - \beta P_{T-2}e^{\mu_Z + \frac{\sigma_Z^2}{2}} (R^2_{T-2} - 2N_{T-2}R_{T-2}) \\
  - \theta N_{T-2}P_{T-2}X_{T-2}(1 - \rho e^{\mu_Z + \frac{\sigma_Z^2}{2}}) - \theta \rho  R_{T-2} P_{T-2} X_{T-2} e^{\mu_Z + \frac{\sigma_Z^2}{2}}$$

Taking the derivative with respect to $N_{T-2}$, we get:

$$ P_{T-2}(1 - e^{\mu_Z + \frac{\sigma_Z^2}{2}}) - 2\beta N_{T-2}P_{T-2}(1 + e^{\mu_Z + \frac{\sigma_Z^2}{2}}) \\
+ 2\beta P_{T-2}R_{T-2}e^{\mu_Z + \frac{\sigma_Z^2}{2}} \\
 - \theta P_{T-2}X_{T-2}(1 - \rho e^{\mu_Z + \frac{\sigma_Z^2}{2}}) = 0 $$

$$ \Rarr N^*_{T-2} = \frac{1 - e^{\mu_Z + \frac{\sigma_Z^2}{2}}}{2\beta(1 + e^{\mu_Z + \frac{\sigma_Z^2}{2}})} + \frac{R_{T-2}e^{\mu_Z + \frac{\sigma_Z^2}{2}}}{1 + e^{\mu_Z + \frac{\sigma_Z^2}{2}}} - \frac{\theta X_{T-2}(1 - \rho e^{\mu_Z + \frac{\sigma_Z^2}{2}})}{2\beta(1 + e^{\mu_Z + \frac{\sigma_Z^2}{2}})} $$

Setting the constant term and coefficients to $ c^{(1)}_{T-2}$, $c^{(2)}_{T-2}$, and $c^{(3)}_{T-2}$ respectively, we get:

$$ N^*_{T-2} =  c^{(1)}_{T-2} + c^{(2)}_{T-2}R_{T-2} + c^{(3)}_{T-2}X_{T-2} $$

Thus, the Value Function becomes:

$$ V((P_{T-2},R_{T-2},X_{T-2})) = P_{T-2}e^{\mu_Z + \frac{\sigma^2_Z}{2}} (N^*_{T-2}(e^{-\mu_Z - \frac{\sigma^2_Z}{2}} - 1) - \beta N^*_{T-2}(e^{-\mu_Z - \frac{\sigma^2_Z}{2}} + 1) \\
- \beta R^2_{T-2} - 2N^*_{T-2}R_{T-2} - \theta N^*_{T-2} X_{T-2}(e^{-\mu_Z - \frac{\sigma^2_Z}{2}} - \rho) - \theta \rho R_{T-2}X_{T-2}) $$

We will denote $(e^{-\mu_Z - \frac{\sigma^2_Z}{2}} - 1) $ as $\alpha$, $(e^{-\mu_Z - \frac{\sigma^2_Z}{2}} + 1) $ as $ \gamma$, and $(e^{-\mu_Z - \frac{\sigma^2_Z}{2}} - \rho) $ as $v$. This gives us:

$$ V_{T-2} = P_{T-2}e^{\mu_Z + \frac{\sigma^2_Z}{2}}(\alpha N^*_{T-2} - \beta \gamma N^*_{T-2} - \beta R^2_{T-2} - 2N^*_{T-2}R_{T-2} \\
 - \theta v N^*_{T-2} X_{T-2} - \theta \rho R_{T-2} X_{T-2}) $$

 $$\Rarr P_{T-2}e^{\mu_Z + \frac{\sigma^2_Z}{2}}(N^*_{T-2}(\alpha - \beta \gamma - 2 R_{T-2} - \theta v X_{T-2}) - \beta R^2_{T-2} - \theta \rho R_{T-2}X_{T-2}) $$

 $$\Rarr  P_{T-2}e^{\mu_Z + \frac{\sigma^2_Z}{2}} ((c^{(1)}_{T-2} + c^{(2)}_{T-2}R_{T-2} + c^{(3)}_{T-2}X_{T-2})(\alpha - \beta \gamma - 2 R_{T-2} - \theta v X_{T-2}) \\
 - \beta R^2_{T-2} - \theta \rho R_{T-2}X_{T-2}) $$

When grouping terms we can assign the coefficients as constants:

$$\Rarr P_{T-2}e^{\mu_Z + \frac{\sigma^2_Z}{2}}(c^{(4)}_{T-2} + c^{(5)}_{T-2} R_{T-2} + c^{(6)}_{T-2} X_{T-2} + c^{(7)}_{T-2}R^2_{T-2} \\
 + c^{(8)}_{T-2} X^2_{T-2} + c^{(9)}_{T-2} R_{T-2} X_{T-2}) $$

Continuing backwards in time, we see that this generalized solution is the same for all $t$:

$$ N^*_t =  c^{(1)}_t + c^{(2)}_t R_t + c^{(3)}_t X_t$$

$$ V^*_t = P_t e^{\mu_Z + \frac{\sigma^2_Z}{2}}(c^{(4)}_t + c^{(5)}_t R_t + c^{(6)}_t X_t + c^{(7)}_t R^2_t \\
 + c^{(8)}_t X^2_t + c^{(9)}_t R_t X_t)

In [23]:
from rl.chapter9.optimal_order_execution import OptimalOrderExecution, PriceAndShares
from rl.function_approx import FunctionApprox, LinearFunctionApprox
from rl.distribution import Gaussian
from rl.policy import DeterministicPolicy
from rl.approximate_dynamic_programming import ValueFunctionApprox
from typing import Tuple, Iterator
from numpy.random import standard_normal
from math import exp

In [ ]:
init_price_mean: float = 100.0
init_price_stdev: float = 10.0
num_shares: int = 100
num_time_steps: int = 5
beta: float = 0.05
theta: float = 0.05
rho: float = 0.05

x = []
x.append(1)
for i in range(1,num_time_steps):
    x.append(rho*x[i-1] + standard_normal())

stand_norm = Gaussian(0,1)



price_diff = [lambda p_s: beta*p_s.shares + theta*x[i] for i in range(num_time_steps)]
dynamics = [lambda p_s: p_s*Gaussian(0,1) for _ in range(num_time_steps)]
ffs = [
    lambda p_s: p_s.state.price * p_s.state.shares,
    lambda p_s: float(p_s.state.shares * p_s.state.shares)
]
fa: FunctionApprox = LinearFunctionApprox.create(feature_functions=ffs)
init_price_distrib: Gaussian = Gaussian(init_price_mean, init_price_stdev)

ooe: OptimalOrderExecution = OptimalOrderExecution(
    shares=num_shares,
    time_steps=num_time_steps,
    avg_exec_price_diff=price_diff,
    price_dynamics=dynamics,
    utility_func=lambda x: x,
    discount_factor=1,
    func_approx=fa,
    initial_price_distribution=init_price_distrib
)
it_vf: Iterator[Tuple[ValueFunctionApprox[PriceAndShares],
                        DeterministicPolicy[PriceAndShares, int]]] = \
    ooe.backward_induction_vf_and_pi()

state: PriceAndShares = PriceAndShares(
    price=init_price_mean,
    shares=num_shares
)
print("Backward Induction: VF And Policy")
print("---------------------------------")
print()
for t, (vf, pol) in enumerate(it_vf):
    print(f"Time {t:d}")
    print()
    opt_sale: int = pol.action_for(state)
    val: float = vf(NonTerminal(state))
    print(f"Optimal Sales = {opt_sale:d}, Opt Val = {val:.3f}")
    print()
    print("Optimal Weights below:")
    print(vf.weights.weights)
    print()


